##Setup

You will need to make a copy of this notebook in your Google Drive before you can edit the homework files. You can do so with **File &rarr; Save a copy in Drive**.

In [ ]:
#@title mount your Google Drive

import os
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
#@title set up mount symlink

DRIVE_PATH = '/content/gdrive/My\ Drive/hw4_part1_16831'
DRIVE_PYTHON_PATH = DRIVE_PATH.replace('\\', '')
if not os.path.exists(DRIVE_PYTHON_PATH):
  %mkdir $DRIVE_PATH

## the space in `My Drive` causes some issues,
## make a symlink to avoid this
SYM_PATH = '/content/hw4_part1_16831'
if not os.path.exists(SYM_PATH):
  !ln -s $DRIVE_PATH $SYM_PATH

In [ ]:
#@title apt install requirements

#@markdown Run each section with Shift+Enter

#@markdown Double-click on section headers to show code.

!apt update
!apt install -y --no-install-recommends \
        build-essential \
        curl \
        git \
        gnupg2 \
        make \
        cmake \
        ffmpeg \
        swig \
        libz-dev \
        unzip \
        zlib1g-dev \
        libglfw3 \
        libglfw3-dev \
        libxrandr2 \
        libxinerama-dev \
        libxi6 \
        libxcursor-dev \
        libgl1-mesa-dev \
        libgl1-mesa-glx \
        libglew-dev \
        libosmesa6-dev \
        lsb-release \
        ack-grep \
        patchelf \
        wget \
        xpra \
        xserver-xorg-dev \
        xvfb \
        python3-opengl \
        ffmpeg > /dev/null 2>&1

In [ ]:
#@title clone homework repo
#@markdown Note that this is the same codebase from homework 1,
#@markdown so you may need to move your old `hw_16831`
#@markdown folder in order to clone the repo again.

#@markdown **Don't delete your old work though!**
#@markdown You will need it for this assignment.

%cd $SYM_PATH
!git clone https://github.com/LeCAR-Lab/16831-S25-HW.git
%cd 16831-S25-HW/hw4
%pip install -r requirements.txt
%pip install -e .

In [ ]:
# downloads mujoco from source
!wget https://mujoco.org/download/mujoco210-linux-x86_64.tar.gz
!tar xzvf mujoco210-linux-x86_64.tar.gz
!mkdir -p ~/.mujoco
!mv mujoco210 ~/.mujoco/mujoco210
!rm mujoco*

%pip install -U mujoco
%pip install -U 'mujoco-py<2.2,>=2.1'
%pip install -U pyvirtualdisplay
%pip install -U gym-notebook-wrapper
%pip install -U "cython<3"

!cp /root/.mujoco/mujoco210/bin/*.so /usr/lib/x86_64-linux-gnu/

In [ ]:
# set env variables
import os
os.environ['LD_LIBRARY_PATH'] += ':/root/.mujoco/mujoco210/bin'
os.environ['MUJOCO_PY_MUJOCO_PATH'] = '/root/.mujoco/mujoco210'
os.environ['LD_LIBRARY_PATH'] += ':/usr/lib/nvidia'

In [ ]:
#@title set up virtual display

from pyvirtualdisplay import Display

display = Display(visible=0, size=(1400, 900))
display.start()

# # For later
# from rob831.hw4_part2.infrastructure.colab_utils import (
#     wrap_env,
#     show_video
# )

In [ ]:
#@title test virtual display

#@markdown If you see a video of a four-legged ant fumbling about, setup is complete!
import numpy as np
np.bool8 = np.bool


import gym
import gnwrapper

env = gnwrapper.LoopAnimation(gym.make('Ant-v4'))

observation = env.reset()
for i in range(100):
    obs, rew, term, _ = env.step(env.action_space.sample())
    env.render()
    if term:
      break

env.display()
env.close()

## Editing Code

To edit code, click the folder icon on the left menu. Navigate to the corresponding file (`hw4_part1_16831/...`). Double click a file to open an editor. There is a timeout of about ~12 hours with Colab while it is active (and less if you close your browser window). We sync your edits to Google Drive so that you won't lose your work in the event of an instance timeout, but you will need to re-mount your Google Drive and re-install packages with every new instance.

## Run Exploration or Exploitation

In [9]:
import os
import time

from rob831.hw4_part2.infrastructure.rl_trainer import RL_Trainer
from rob831.hw4_part2.agents.explore_or_exploit_agent import ExplorationOrExploitationAgent
from rob831.hw4_part2.infrastructure.dqn_utils import get_env_kwargs, PiecewiseSchedule, ConstantSchedule

%load_ext autoreload
%autoreload 2

ModuleNotFoundError: No module named 'tensorboardX'

In [ ]:
#@title runtime arguments

class Args:

  def __getitem__(self, key):
    return getattr(self, key)

  def __setitem__(self, key, val):
    setattr(self, key, val)

  def __contains__(self, key):
    return hasattr(self, key)

  env_name = "PointmassEasy-v0" #@param ["PointmassEasy-v0", "PointmassMedium-v0", "PointmassHard-v0"]
  exp_name = 'temp'#@param {type: "string"}

  #@markdown batches and steps
  batch_size = 256 #@param {type: "integer"}
  eval_batch_size = 1000 #@param {type: "integer"}

  #@markdown exploration hyperparameters
  use_rnd = False #@param {type: "boolean"}
  unsupervised_exploration = False #@param {type: "boolean"}
  num_exploration_steps = 10000 #@param {type: "integer"}

  #@markdown offline training hyperparameters
  offline_exploitation = False #@param {type: "boolean"}
  cql_alpha = 0.0 #@param {type: "raw"}

  #@markdown reward shifting hyperparameters
  exploit_rew_shift = 0.0 #@param {type: "raw"}
  exploit_rew_scale = 1.0 #@param {type: "raw"}

  #@markdown exploration model hyperparameters
  rnd_output_size = 5 #@param {type: "integer"}
  rnd_n_layers = 2 #@param {type: "integer"}
  rnd_size = 400 #@param {type: "integer"}

  #@markdown experiment hyperparameters
  seed = 1 #@param {type: "integer"}
  no_gpu = False #@param {type: "boolean"}
  which_gpu = 0 #@param {type: "integer"}
  scalar_log_freq = 1000 #@param {type: "integer"}
  save_params = False #@param {type: "boolean"}


args = Args()

## ensure compatibility with hw1 code
args['train_batch_size'] = args['batch_size']

In [ ]:
###### HARDCODED DETAILS, DO NOT CHANGE ######

args['video_log_freq'] = -1 # Not used
args['exploit_weight_schedule'] = ConstantSchedule(1.0)
args['num_agent_train_steps_per_iter'] = 1
args['num_critic_updates_per_agent_update'] = 1
args['learning_starts'] = 2000
args['num_timesteps'] = 50000
args['double_q'] = True
args['eps'] = 0.2

if args['env_name']=='PointmassEasy-v0':
  args['ep_len']=50
if args['env_name']=='PointmassMedium-v0':
  args['ep_len']=150
if args['env_name']=='PointmassHard-v0':
  args['ep_len']=100
if args['env_name']=='PointmassVeryHard-v0':
  args['ep_len']=200

if args['use_rnd']:
  args['explore_weight_schedule'] = PiecewiseSchedule([(0,1), (args['num_exploration_steps'], 0)], outside_value=0.0)
else:
  args['explore_weight_schedule'] = ConstantSchedule(0.0)

if args['unsupervised_exploration']:
  args['explore_weight_schedule'] = ConstantSchedule(1.0)
  args['exploit_weight_schedule'] = ConstantSchedule(0.0)

  if not args['use_rnd']:
    args['learning_starts'] = args['num_exploration_steps']

In [ ]:
#@title create directories for logging

data_path = '''/content/hw4_part1_16831/''' \
        '''homework_spring2024/hw4/data'''

if not (os.path.exists(data_path)):
    os.makedirs(data_path)

logdir = 'hw4_' + args.exp_name + '_' + args.env_name + '_' + time.strftime("%d-%m-%Y_%H-%M-%S")
logdir = os.path.join(data_path, logdir)
args['logdir'] = logdir
if not(os.path.exists(logdir)):
    os.makedirs(logdir)

print("LOGGING TO: ", logdir)


In [ ]:
#@title Define Exploration Agent

class Q_Trainer(object):

    def __init__(self, params):
        self.params = params

        train_args = {
            'num_agent_train_steps_per_iter': params['num_agent_train_steps_per_iter'],
            'num_critic_updates_per_agent_update': params['num_critic_updates_per_agent_update'],
            'train_batch_size': params['batch_size'],
            'double_q': params['double_q'],
        }

        env_args = get_env_kwargs(params['env_name'])

        for k, v in env_args.items():
          params[k] = v
        for k, v in train_args.items():
          params[k] = v

        self.agent_params = params

        self.params['agent_class'] = ExplorationOrExploitationAgent
        self.params['agent_params'] = self.agent_params
        self.params['train_batch_size'] = params['batch_size']
        self.params['env_wrappers'] = self.agent_params['env_wrappers']

        self.rl_trainer = RL_Trainer(self.params)

    def run_training_loop(self):
        self.rl_trainer.run_training_loop(
            self.agent_params['num_timesteps'],
            collect_policy = self.rl_trainer.agent.actor,
            eval_policy = self.rl_trainer.agent.actor,
            )


In [ ]:
#@title run training

trainer = Q_Trainer(args)
trainer.run_training_loop()

In [ ]:
#@markdown You can visualize your runs with tensorboard from within the notebook

## requires tensorflow==2.3.0
%load_ext tensorboard
%tensorboard --logdir /content/hw4_part1_16831/homework_spring2024/hw4/data/